# Flight Delays Data Analysis - Initial Exploration

This notebook contains the initial exploration of flight delays data from both CSV files and API sources. We'll go through the process of loading, cleaning, and performing basic analysis of the data.

## Import Required Libraries

First, let's import all the necessary libraries for our analysis.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import duckdb

# API requests
import requests

# Utilities
from pathlib import Path
from tqdm.auto import tqdm

## Data Loading
This section will cover methods to import the flight delay data from CSVs into a pandas dataframe. First, we being by creating some constant variables for easy file path referencing.

In [2]:
# Define directory paths
DATA_DIR = Path.cwd().parent / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

In [3]:
# Create list of file paths for each CSV
file_paths = [file_name for file_name in sorted(RAW_DATA_DIR.iterdir()) if ".csv" == file_name.suffix]

### Loading CSVs Directly & Then Filtering
This was my first naive approach to loading and filtering the flight delay data. **This method will not work in a google colab notebook because loading each dataset fully into memory eats up too much RAM**.

In [ ]:
%%time

# Create an empty df, iterate through paths, load data into df, 
# filter df for PHL, concat to originally empty df and overwrite.
data_raw = pd.DataFrame()
for path in file_paths:
    print(f"----- Appending {path.name} -----")
    
    data_new = pd.read_csv(path)
    print(f"{path.name} full data memory usage: {data_new.memory_usage(deep = True).sum()} bytes.")
    data_filtered = data_new[data_new["ORIGIN"] == "PHL"]
    print(f"Filtered {path.name} data memory usage: {data_filtered.memory_usage(deep = True).sum()} bytes.")
    data_raw = pd.concat([data_raw, data_filtered])
    print(f"Raw data after {path.name} memory usage: {data_raw.memory_usage(deep = True).sum()} bytes.")
    print("")

    
print("Done!")

----- Appending 2009.csv -----
2009.csv full data memory usage: 2766182156 bytes.
Filtered 2009.csv data memory usage: 40697706 bytes.
Raw data after 2009.csv memory usage: 40697706 bytes.

----- Appending 2010.csv -----
2010.csv full data memory usage: 2775589032 bytes.
Filtered 2010.csv data memory usage: 38854980 bytes.
Raw data after 2010.csv memory usage: 79552686 bytes.

----- Appending 2011.csv -----
2011.csv full data memory usage: 2610748280 bytes.
Filtered 2011.csv data memory usage: 37716360 bytes.
Raw data after 2011.csv memory usage: 117269046 bytes.

----- Appending 2012.csv -----
2012.csv full data memory usage: 2623027308 bytes.
Filtered 2012.csv data memory usage: 34413126 bytes.
Raw data after 2012.csv memory usage: 151682172 bytes.

----- Appending 2013.csv -----
2013.csv full data memory usage: 2740605608 bytes.
Filtered 2013.csv data memory usage: 35495742 bytes.
Raw data after 2013.csv memory usage: 187177914 bytes.

Done!
CPU times: user 33.5 s, sys: 4.25 s, tota

### Load CSVs With Filtered Streaming
This approach worked inside of Google Colab. With this method, the only data loaded into memory is a 100,000 row chunk of the CSV. This chunk is then filtered before being appended to a list. This prevents us from loading the entire CSV into memory.

In [16]:
%%time

# Chunk each CSV, filter the chunk and load into memory,
# append to a list, concatenate list of filtered chunks together
data_to_concat = []

for path in (pbar := tqdm(file_paths)):
    pbar.set_description(f"Processing & Adding {path.name}")
    chunks = pd.read_csv(path, chunksize=100_000, low_memory = False)
    
    # For each chunk, filter for just PHL, add to concat list
    chunked_total = 0
    
    for chunk in tqdm(chunks, desc = f"Processing chunks from {path}", leave = False):
        # This is where chunk is loaded into memory
        chunk_filtered = chunk[chunk["ORIGIN"] == "PHL"]
        chunked_total += chunk_filtered.memory_usage(deep = True).sum()
        
        if not chunk_filtered.empty:
            data_to_concat.append(chunk_filtered)
    
    print(f"{path.name} chunked & filtered data memory usage: {chunked_total} bytes.")

print("")
data_raw = pd.concat(data_to_concat, ignore_index=True)
print(f"Raw data total memory usage: {data_raw.memory_usage(deep = True).sum()} bytes.")
print("")

  0%|          | 0/5 [00:00<?, ?it/s]

Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2009.csv: 0it […

2009.csv chunked & filtered data memory usage: 40697706 bytes.


Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2010.csv: 0it […

2010.csv chunked & filtered data memory usage: 38854980 bytes.


...

2011.csv chunked & filtered data memory usage: 37716360 bytes.


Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2012.csv: 0it […

2012.csv chunked & filtered data memory usage: 34413126 bytes.


Processing chunks from /Users/rohankrishnan/Documents/GitHub/flight-delays-prediction/data/raw/2013.csv: 0it […

2013.csv chunked & filtered data memory usage: 35495742 bytes.

Raw data total memory usage: 183761910 bytes.

CPU times: user 27.3 s, sys: 3.96 s, total: 31.3 s
Wall time: 31.4 s


### Load CSVs With DuckDB
This approach should be the fastest. DuckDB allows us to query the data before loading it into memory so we only load the filtered data. I did this one last because in order to use `read_csv_auto` without error, we need to pass a schema to teach DuckDB the dtypes for each column.

In [38]:
%%time

# Create schema dict using dtype_map to translate pandas dtypes to SQL dtypes
# If we were to just use duckdb from the get-go, this would probably need to be
# made by hand
schema = {}
dtype_map = {
    "object": "VARCHAR",
    "int64": "BIGINT",
    "float64": "DOUBLE",
    "bool": "BOOLEAN",
    "datetime64[ns]": "TIMESTAMP"
}
for column in data_raw.columns:
    schema[column] = dtype_map[str(data_raw[column].dtype)]

# DuckDB connection, make data view, query data and store as df
conn = duckdb.connect()

data_view = duckdb.read_csv(
    f"{RAW_DATA_DIR}/*.csv",
    columns = schema,
    auto_detect = True
)

data_raw = duckdb.sql("""
    SELECT *
    FROM data_view
    WHERE ORIGIN = 'PHL';
"""   
).df()

print(f"DuckDB raw data total memory usage: {data_raw.memory_usage(deep = True).sum()} bytes.")
print("")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

DuckDB raw data total memory usage: 180409982 bytes.

CPU times: user 26.5 s, sys: 955 ms, total: 27.5 s
Wall time: 3.35 s


In [53]:
data_raw.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,2248,PHL,CLE,1312.0,1311.0,-1.0,9.0,1320.0,...,92.0,78.0,64.0,363.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,XE,2545,PHL,CLE,1750.0,1801.0,11.0,18.0,1819.0,...,98.0,87.0,62.0,363.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,YV,7144,PHL,IAD,1010.0,958.0,-12.0,17.0,1015.0,...,83.0,57.0,34.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,YV,7218,PHL,IAD,1910.0,1910.0,0.0,12.0,1922.0,...,69.0,52.0,32.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,NW,1761,PHL,DTW,2000.0,1949.0,-11.0,14.0,2003.0,...,120.0,95.0,77.0,453.0,NaN,NaN,NaN,NaN,NaN,NaN


# Weather Data API Call
This section covers how to call the OpenMeteo API and construct a dataframe of weather data.

In [ ]:
# Define variables using the data & API Website variable selector
date_start, date_end = min(pd.to_datetime(data_raw["FL_DATE"])), max(pd.to_datetime(data_raw["FL_DATE"]))

daily_vars = [
    "temperature_2m_mean",
    "temperature_2m_max",
    "temperature_2m_min",
    "apparent_temperature_mean",
    "apparent_temperature_max",
    "apparent_temperature_min",
    "wind_speed_10m_max",
    "wind_gusts_10m_max",
    "wind_direction_10m_dominant",
    "shortwave_radiation_sum",
    "et0_fao_evapotranspiration",
    "precipitation_sum",
    "rain_sum",
    "snowfall_sum",
    "precipitation_hours",
    "weather_code",
]

In [ ]:
# Call API with params
API_URL = "https://archive-api.open-meteo.com/v1/archive"


params = {
    "latitude": 38.87, # Googled
    "longitude": 75.24, # Googled
    "start_date": date_start.strftime("%Y-%m-%d"),
    "end_date": date_end.strftime("%Y-%m-%d"),
    "daily": daily_vars
}

response = requests.get(API_URL, params = params)
print(response.status_code) # Check that code is 200

200


In [ ]:
# Extract daily weather data from json response and store as a pandas df
data_weather = pd.DataFrame(response.json()["daily"])

# Joining
This section covers joining the raw delay data with the weather data pulled from the OpenMeteo API.

In [48]:
# Make copies of each df before making edits
delays_df = data_raw.copy()
weather_df = data_weather.copy()

# Standardize date col types
delays_df["FL_DATE"] = pd.to_datetime(delays_df["FL_DATE"])
weather_df["time"] = pd.to_datetime(weather_df["time"])

In [ ]:
# Left join delays_df with weather_df on respective date columns
combined_df = pd.merge(left = delays_df, right = weather_df,
                     how = "left", left_on = "FL_DATE", 
                     right_on = "time")

In [52]:
combined_df.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,weather_code
0,2009-01-01,XE,2248,PHL,CLE,1312.0,1311.0,-1.0,9.0,1320.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
1,2009-01-01,XE,2545,PHL,CLE,1750.0,1801.0,11.0,18.0,1819.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
2,2009-01-01,YV,7144,PHL,IAD,1010.0,958.0,-12.0,17.0,1015.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
3,2009-01-01,YV,7218,PHL,IAD,1910.0,1910.0,0.0,12.0,1922.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3
4,2009-01-01,NW,1761,PHL,DTW,2000.0,1949.0,-11.0,14.0,2003.0,...,8.7,45.7,225,10.84,0.52,0.0,0.0,0.0,0.0,3


The code and text above walks through the steps to load and combine the data. Next steps would be cleaning the data, engineering other features, and/or gathering other datasets to include in our analysis.